In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-5b73  GCE       2                                       RUNNING  us-central1-c


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 22 20:58 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '318932639' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [8]:
paths = [p for p in paths if p.endswith('parquet')]

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [9]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

In [10]:
doc_title_pairs = parquetFile.select("title", "id").rdd

In [11]:
# doc_anchor_pairs = parquetFile.select("anchor_text").rdd
# doc_anchor_pairs = doc_anchor_pairs.flatMap(lambda x: x)
# doc_anchor_pairs = doc_anchor_pairs.flatMap(lambda x: x)
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [12]:
# Count number of wiki pages
parquetFile.count()

6348910

In [13]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [14]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [15]:
from inverted_index_gcp import InvertedIndex

In [18]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
from functools import partial
def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in 
    `all_stopwords` and return entries that will go into our posting lists. 
    Parameters:
    -----------
    text: str
      Text of one document
    id: int
      Document id
    Returns:
    --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    from collections import Counter

    tokens_without_stopwords = [token for token in tokens if token not in all_stopwords]
    tokens_count = Counter(tokens_without_stopwords)

    return [(token, (id, count)) for token, count in tokens_count.items()]

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
    Returns:
    --------
    list of tuples
      A sorted posting list.
    '''
    return sorted(unsorted_pl, key=lambda x: x[0])

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each element is a (token, df) pair.
    '''
    return postings.mapValues(lambda pl: len(pl))

def partition_postings_and_write(postings, dir_name):
    ''' A function that partitions the posting lists into buckets, writes out 
    all posting lists in a bucket to disk, and returns the posting locations for 
    each bucket. Partitioning should be done through the use of `token2bucket` 
    above. Writing to disk should use the function  `write_a_posting_list`, a 
    static method implemented in inverted_index_colab.py under the InvertedIndex 
    class. 
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
    '''
    write_a_posting_list = partial(InvertedIndex.write_a_posting_list, bucket_name = bucket_name, dir_name = dir_name)
    return postings.map(lambda x:(token2bucket_id(x[0]), x)).groupByKey().map(write_a_posting_list)

## Body Index

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered, dir_name='postings_gcp_text').collect()
index_const_time = time() - t_start

In [ ]:
index_const_time

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_text'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
        super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_text/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

## Title Index

In [19]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered, dir_name='postings_gcp_title').collect()
index_const_time = time() - t_start

In [20]:
index_const_time

112.00684356689453

In [21]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_title'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
        super_posting_locs[k].extend(v)

Putting it all together

In [22]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_title/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][953.6 KiB/953.6 KiB]                                                
Operation completed over 1 objects/953.6 KiB.                                    


In [23]:
!gsutil ls -lh $index_dst

953.59 KiB  2023-01-22T21:16:16Z  gs://318932639/postings_gcp_title/index.pkl
TOTAL: 1 objects, 976479 bytes (953.59 KiB)


## Anchor Index

In [24]:
def parse_anchor(pair):
    lst = []
    for tup in pair[0]:
        lst.append((tup[1], tup[0]))
    return lst

tmp_pairs = doc_anchor_pairs.flatMap(lambda pair: parse_anchor(pair))

word_counts_anchor = tmp_pairs.flatMap(lambda x: word_count(x[0],x[1]))
posting_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)

In [25]:
def group(lst):
    counter = {}
    for tup in lst:
        if tup[0] not in counter:
            counter[tup[0]] = 0
        counter[tup[0]] += 1
    return [(key, counter[key]) for key in counter.keys()]
grouped_postings = posting_anchor.map(lambda x: (x[0], group(x[1])))

In [ ]:
# filtering postings and calculate df
w2df = calculate_df(grouped_postings)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(grouped_postings, dir_name='postings_gcp_anchor').collect()

In [ ]:
# # time the index creation time
# t_start = time()
# # word counts map
# word_counts = doc_anchor_pairs.flatMap(lambda x: word_count(x[1], x[0]))
# postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# # filtering postings and calculate df
# postings_filtered = postings.filter(lambda x: len(x[1])>50)
# w2df = calculate_df(postings_filtered)
# w2df_dict = w2df.collectAsMap()
# # partition posting lists and write out
# _ = partition_postings_and_write(postings_filtered).collect()
# index_const_time = time() - t_start

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_anchor'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
        super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_anchor/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 95.0 MiB/ 95.0 MiB]                                                
Operation completed over 1 objects/95.0 MiB.                                     


In [ ]:
!gsutil ls -lh $index_dst

 95.03 MiB  2023-01-22T22:04:40Z  gs://318932639/postings_gcp_anchor/index.pkl
TOTAL: 1 objects, 99650317 bytes (95.03 MiB)


# PageRank

In [ ]:
def generate_graph(pages):
    ''' Compute the directed graph generated by wiki links.
    Parameters:
    -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
    Returns:
    --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
    '''
    # YOUR CODE HERE
    edges = pages.flatMap(lambda row:
                        [(row[0], anchor[0]) for anchor in row[1]]).distinct()
    vertices = edges.flatMap(lambda x: x).distinct()
    vertices = vertices.map(lambda x: [x])

    return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# pages_links = parquetFile.select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()